In [1]:
import numpy as np
import cvxpy
import scipy
import scs

In [2]:
print(scs.__version__)
assert(int(scs.__version__[0]) >= 2)

3.0.0


In [3]:
# Robust Gaussian covariance estimation
#np.random.seed(hash('rob-cov-var') % 2 ** 31)

m = 200 # Number of observations of each random variable
n = 5 # Number of random variables
k = 3 # Needs to be less than m.

A = np.matrix(np.random.rand(m, n))
A -= np.mean(A, axis=0)
K = np.array([list((A[i].T @ A[i]).flat) for i in range(m)])

sigma_inv1 = cvxpy.Variable((n, n), PSD=True) # Inverse covariance matrix
#t = cvxpy.Variable(m)
tdet = cvxpy.Variable(1)

obj = cvxpy.sum_largest(K @ cvxpy.reshape(sigma_inv1, n * n) + tdet, k)
constraints = [-cvxpy.log_det(sigma_inv1) <= tdet, sigma_inv1 >> 0]
prob = cvxpy.Problem(cvxpy.Minimize(obj), constraints)
prob.solve(solver='SCS', verbose=True)
#prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)
#prob.solve(verbose=True)


------------------------------------------------------------------
	       SCS v3.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 237, constraints m: 501
cones: 	  l: linear vars: 401
	  s: sd vars: 85, sd blks: 3
	  e: exp vars: 15, dual exp vars: 0
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-05
	  alpha: 1.50, scale: 1.00e+00, adaptive_scaling: 1
	  max_iters: 100000, normalize: 1, warm_start: 0
lin-sys:  sparse-direct
	  nnz(A): 3881, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 1.47e+01  3.00e+00  5.12e+02 -5.05e+02  1.00e+00  3.15e-03 
   250| 2.04e-03  4.08e-03  1.64e-03 -1.46e+01  1.00e+00  3.58e-02 
   450| 1.32e-04  5.66e-04  1.99e-04 -1.46e+01  1.00e+00  6.03e-02 
--

-inf

In [4]:
data = prob.get_problem_data(cvxpy.SCS)[0]
cone = {'f': data['dims'].zero, 'l': data['dims'].nonneg, 'ep': data['dims'].exp, 's':data['dims'].psd}
results = scs.solve(data, cone, eps_infeas=1e-9)

------------------------------------------------------------------
	       SCS v3.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 237, constraints m: 501
cones: 	  l: linear vars: 401
	  s: sd vars: 85, sd blks: 3
	  e: exp vars: 15, dual exp vars: 0
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-09
	  alpha: 1.50, scale: 1.00e+00, adaptive_scaling: 1
	  max_iters: 100000, normalize: 1, warm_start: 0
lin-sys:  sparse-direct
	  nnz(A): 3881, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 1.47e+01  3.00e+00  5.12e+02 -5.05e+02  1.00e+00  2.02e-03 
   250| 2.04e-03  4.08e-03  1.64e-03 -1.46e+01  1.00e+00  3.68e-02 
   475| 2.11e-04  3.87e-04  2.17e-04 -1.46e+01  1.00e+00  6.77e-02 
--

In [5]:
print(results['x'].T @ data['c'])
print(np.linalg.norm(data['A'] @ results['x'] + results['s'] - data['b'], np.inf))
print(np.linalg.norm(data['A'].T @ results['y'] + data['c'], np.inf))
print(data['c'].T @ results['x']+data['b'].T @ results['y'])
print(results['s'] @ results['y'] / np.linalg.norm(results['s'], np.inf) / np.linalg.norm(results['y'], np.inf))

-14.616120517094428
0.00021128017340288352
0.00038660362153020156
-0.00021711050033523804
-9.708464962119129e-11


In [8]:
data = prob.get_problem_data(cvxpy.SCS)[0]
cone = {'f': data['dims'].zero, 'l': data['dims'].nonneg, 'ep': data['dims'].exp, 's':data['dims'].psd}
data['x'] = results['x']
data['y'] = results['y']
data['s'] = results['s']

scs.solve(data, cone, normalize=True, eps_infeas=1e-9)

------------------------------------------------------------------
	       SCS v3.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 237, constraints m: 501
cones: 	  l: linear vars: 401
	  s: sd vars: 85, sd blks: 3
	  e: exp vars: 15, dual exp vars: 0
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-09
	  alpha: 1.50, scale: 1.00e+00, adaptive_scaling: 1
	  max_iters: 100000, normalize: 1, warm_start: 1
lin-sys:  sparse-direct
	  nnz(A): 3881, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 2.04e-04  3.95e-04  6.13e-04 -1.46e+01  1.00e+00  1.88e-03 
------------------------------------------------------------------
status:  solved
timings: total: 5.75e-03s = setup: 3.85e-03s + solve: 1

{'x': array([-7.33232342e-07, -1.41531432e-06, -2.28286104e-06, -1.10948550e-06,
        -1.33928324e-06,  1.68929118e-07, -8.96200363e-07, -1.06962777e-06,
        -1.02432674e-06, -8.94306115e-07, -1.83712422e-06, -8.02582785e-07,
        -7.79686633e-07, -9.66192150e-07, -2.86561805e-06, -1.16706464e-06,
        -3.52049025e-07, -1.21344184e-06, -1.58866190e-06, -1.65529555e-06,
        -2.56263263e-06, -2.96413496e-06, -1.07348179e-06, -2.24869824e-06,
        -3.92152744e-07, -7.51472668e-07, -9.19380509e-07, -9.14885935e-07,
        -1.38573583e-06, -8.33995648e-07, -8.71624616e-07, -7.15965006e-07,
        -7.39550476e-07, -1.78057438e-06, -1.37916082e-06, -1.48934089e-06,
        -1.46547119e-06, -4.68502759e-07, -9.01656589e-07, -1.75483355e-06,
        -1.59460380e-06, -1.07927537e-06, -9.09985462e-07, -3.33651539e-07,
        -1.26714510e-06, -3.82318396e-07, -2.82933117e-06, -9.69887959e-07,
        -3.64219341e-07, -7.76008919e-07, -6.58406900e-07, -1.70857372e-06,
       

In [3]:
# Random LP
np.random.seed(hash('lp') % 2 ** 30)

# Dimensions
n = 100
m = 70

A = scipy.sparse.random(m, n, density=0.2, data_rvs = np.random.randn)
b = np.random.randn(m)
c = np.random.rand(n)

# Problem construction
x = cvxpy.Variable(n)
objective = cvxpy.Minimize(c.T @ x)
constraints = [x >= 0, A @ x <= b]
prob = cvxpy.Problem(objective, constraints)

prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

                                     CVXPY                                     
                                    v1.1.12                                    
(CVXPY) May 19 12:47:14 AM: Your problem has 100 variables, 2 constraints, and 0 parameters.
(CVXPY) May 19 12:47:14 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 19 12:47:14 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 19 12:47:14 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 19 12:47:14 AM: Compiling problem (target solver=SCS).
(CVXPY) May 19 12:47:14 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing ->

12.676277704951103

In [10]:
# Optimal control
np.random.seed(hash('opt-control') % 2 ** 31)

# Problem data
T = 10
n, p = (10, 5)
A = np.random.randn(n, n)
B = np.random.randn(n, p)
x_init = np.random.randn(n)
x_final = np.random.randn(n)

def step(A, B, x_prev):
    x = cvxpy.Variable(n)
    u = cvxpy.Variable(p)
    cost = sum(cvxpy.square(u)) + sum(cvxpy.abs(x))
    constraint = (x == A @ x_prev + B @ u)
    return cost, constraint, x

x = cvxpy.Variable(n)
constraints = [(x == x_init)]
total_cost = 0.
for t in range(T):
    cost_t, constraint_t, x = step(A, B, x)
    constraints.append(constraint_t)
    total_cost += cost_t

constraints.append(x == x_final)
prob = cvxpy.Problem(cvxpy.Minimize(total_cost), constraints)
prob.solve(solver='SCS', verbose=True)
#prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

------------------------------------------------------------------
	       SCS v3.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 1410, constraints m: 2870
cones: 	  f: primal zero / dual free vars: 120
	  l: linear vars: 2000
	  q: soc vars: 750, soc blks: 250
settings: eps_abs: 1.0e-04, eps_rel: 1.0e-04, eps_infeas: 1.0e-05
	  alpha: 1.50, scale: 1.00e+00, adaptive_scaling: 1
	  max_iters: 100000, normalize: 1, warm_start: 0
lin-sys:  sparse-direct
	  nnz(A): 6370, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 9.69e+00  6.24e+01  6.30e+03 -2.34e+02  1.00e+00  1.80e-03 
   250| 2.73e-03  1.00e-02  2.53e-03  2.93e+02  1.00e+00  2.19e-02 
   500| 5.04e-04  2.07e-04  4.32e-05  2.93e+02  1.00e+00  4

292.64946998506883

In [18]:
data = prob.get_problem_data(cvxpy.SCS)[0]
cone = {'f': data['dims'].zero, 'l': data['dims'].nonneg, 'q': data['dims'].soc, 'ep': data['dims'].exp, 's':data['dims'].psd}
results = scs.solve(data, cone, eps_infeas=1e-9, eps_abs=1e-9, eps_rel=1e-9)

------------------------------------------------------------------
	       SCS v3.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 1410, constraints m: 2870
cones: 	  f: primal zero / dual free vars: 120
	  l: linear vars: 2000
	  q: soc vars: 750, soc blks: 250
settings: eps_abs: 1.0e-09, eps_rel: 1.0e-09, eps_infeas: 1.0e-09
	  alpha: 1.50, scale: 1.00e+00, adaptive_scaling: 1
	  max_iters: 100000, normalize: 1, warm_start: 0
lin-sys:  sparse-direct
	  nnz(A): 6370, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 9.69e+00  6.24e+01  6.30e+03 -2.34e+02  1.00e+00  2.54e-03 
   250| 2.73e-03  1.00e-02  2.53e-03  2.93e+02  1.00e+00  2.85e-02 
   500| 5.04e-04  2.07e-04  4.32e-05  2.93e+02  1.00e+00  5

In [19]:
data = prob.get_problem_data(cvxpy.SCS)[0]
cone = {'f': data['dims'].zero, 'l': data['dims'].nonneg, 'q': data['dims'].soc, 'ep': data['dims'].exp, 's':data['dims'].psd}
data['x'] = results['x']
data['y'] = results['y']
data['s'] = results['s']

scs.solve(data, cone, normalize=True, eps_abs=1e-9, eps_rel=1e-9)

------------------------------------------------------------------
	       SCS v3.0.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 1410, constraints m: 2870
cones: 	  f: primal zero / dual free vars: 120
	  l: linear vars: 2000
	  q: soc vars: 750, soc blks: 250
settings: eps_abs: 1.0e-09, eps_rel: 1.0e-09, eps_infeas: 1.0e-05
	  alpha: 1.50, scale: 1.00e+00, adaptive_scaling: 1
	  max_iters: 100000, normalize: 1, warm_start: 1
lin-sys:  sparse-direct
	  nnz(A): 6370, nnz(P): 0
------------------------------------------------------------------
 iter | pri res | dua res |   gap   |   obj   |  scale  | time (s)
------------------------------------------------------------------
     0| 7.07e-11  1.93e-09  3.71e-09  2.93e+02  1.00e+00  2.55e-03 
------------------------------------------------------------------
status:  solved
timings: total: 7.94e-03s = setup: 5.35e-03s

{'x': array([41.53188288, 46.04786662, 16.46566084, ..., -0.77070807,
         1.0679389 ,  1.35572501]),
 'y': array([-44.371373  , -45.40604295, -31.73648637, ...,   0.52685965,
          0.47314035,   0.23177426]),
 's': array([ 0.        ,  0.        ,  0.        , ...,  1.05371931,
        -0.94628069, -0.46354852]),
 'info': {'statusVal': 1,
  'iter': 0,
  'scale_updates': 0,
  'scale': 1.0,
  'pobj': 292.6511685582295,
  'dobj': 292.6511685619429,
  'resPri': 7.06739310911407e-11,
  'resDual': 1.9326854042363324e-09,
  'gap': 3.713371054503181e-09,
  'resInfeas': 1.0000000001932134,
  'resUnbddA': 2.426221061062196,
  'resUnbddP': 0.0,
  'solveTime': 2.591217,
  'setupTime': 5.353682,
  'status': 'solved'}}

In [5]:
# Lasso
np.random.seed(hash('lasso') % 2 ** 31)

# Dimensions
n = 100
m = 50

x_true = scipy.sparse.rand(n, 1, density=0.1) 
A = scipy.sparse.random(m, n, density=0.2, data_rvs = np.random.randn)
b = A @ x_true + 0.1 * np.random.randn(m, 1)
b = np.array([x for x in b.flat])
mu = 1

# Problem construction
x = cvxpy.Variable(n)
objective = cvxpy.Minimize(0.5 * cvxpy.sum_squares(A @ x - b) + mu * cvxpy.norm1(x))
prob = cvxpy.Problem(objective)

prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

                                     CVXPY                                     
                                    v1.1.12                                    
(CVXPY) May 19 12:47:16 AM: Your problem has 100 variables, 0 constraints, and 0 parameters.
(CVXPY) May 19 12:47:16 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 19 12:47:16 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 19 12:47:16 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 19 12:47:16 AM: Compiling problem (target solver=SCS).
(CVXPY) May 19 12:47:16 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing ->

5.014983290865015

In [6]:
# Nonnegative Lasso
np.random.seed(hash('nonneg-lasso') % 2 ** 31)

# Dimensions
n = 100
m = 50

x_true = scipy.sparse.rand(n, 1, density=0.1) 
A = scipy.sparse.random(m, n, density=0.2, data_rvs = np.random.randn)
b = A @ x_true + 0.1 * np.random.randn(m, 1)
b = np.array([x for x in b.flat])
mu = 1

# Problem construction
x = cvxpy.Variable(n)
objective = cvxpy.Minimize(0.5 * cvxpy.sum_squares(A @ x - b) + mu * cvxpy.norm1(x))
constraints = [x >= 0]
prob = cvxpy.Problem(objective, constraints)

prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

                                     CVXPY                                     
                                    v1.1.12                                    
(CVXPY) May 19 12:47:16 AM: Your problem has 100 variables, 1 constraints, and 0 parameters.
(CVXPY) May 19 12:47:16 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 19 12:47:16 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 19 12:47:16 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 19 12:47:16 AM: Compiling problem (target solver=SCS).
(CVXPY) May 19 12:47:16 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing ->

4.987373189522275

In [7]:
# SDP for closest elemwise-positive PSD matrix in some metric
np.random.seed(hash('sdp') % 2 ** 31)

# create data P
n = 50
P = np.random.randn(n, n)
P = P + P.T
Z = cvxpy.Variable((n, n), PSD=True)

objective = cvxpy.Maximize(cvxpy.lambda_min(P - Z))
prob = cvxpy.Problem(objective, [Z >= 0])
prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

                                     CVXPY                                     
                                    v1.1.12                                    
(CVXPY) May 19 12:47:16 AM: Your problem has 2500 variables, 1 constraints, and 0 parameters.
(CVXPY) May 19 12:47:16 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 19 12:47:16 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 19 12:47:16 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 19 12:47:16 AM: Compiling problem (target solver=SCS).
(CVXPY) May 19 12:47:16 AM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> Con

-18.0039122559315

In [8]:
# Basis pursuit
np.random.seed(hash('basis-pursuit') % 2 ** 31)

n = 300
m = 100
x = cvxpy.Variable(n)
A = np.random.rand(m, n)
x0 = scipy.sparse.rand(n, 1, 0.1)
b = A @ x0
b = np.array([x for x in b.flat])

prob = cvxpy.Problem(cvxpy.Minimize(cvxpy.norm1(x)), [A @ x == b])
prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

                                     CVXPY                                     
                                    v1.1.12                                    
(CVXPY) May 19 12:47:16 AM: Your problem has 300 variables, 1 constraints, and 0 parameters.
(CVXPY) May 19 12:47:16 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 19 12:47:16 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 19 12:47:16 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 19 12:47:16 AM: Compiling problem (target solver=SCS).
(CVXPY) May 19 12:47:16 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing ->

18.28266703962421

In [9]:
# Chebyshev
np.random.seed(hash('chebyshev') % 2 ** 31)

def normalized_data_matrix(m, n):
    A = np.random.randn(m, n)
    A /= np.sqrt(np.sum(A**2, 0))
    return A

m = 100
n = 200
k = 50
A = [normalized_data_matrix(m, n) for i in range(k)]
B = normalized_data_matrix(k, n)
c = np.random.rand(k)

x = cvxpy.Variable(n)
t = cvxpy.Variable(k)

f = cvxpy.max(t + cvxpy.abs(B @ x - c))
constraints = []
for i in range(k):
    constraints.append(cvxpy.norm(A[i] @ x) <= t[i])

prob = cvxpy.Problem(cvxpy.Minimize(f), constraints)
prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

                                     CVXPY                                     
                                    v1.1.12                                    
(CVXPY) May 19 12:47:17 AM: Your problem has 250 variables, 50 constraints, and 0 parameters.
(CVXPY) May 19 12:47:17 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 19 12:47:17 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 19 12:47:17 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 19 12:47:17 AM: Compiling problem (target solver=SCS).
(CVXPY) May 19 12:47:17 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -

0.9177073712433654

In [10]:
# Least absolute deviation
np.random.seed(hash('least-abs-dev') % 2 ** 31)

m = 5000
n = 200

A = np.random.randn(m,n);
b = A.dot(np.random.randn(n))

# Add outlier noise.
k = int(0.02 * m)
idx = np.random.randint(m, size=k)
b[idx] += 10 * np.random.randn(k)

x = cvxpy.Variable(n)

prob = cvxpy.Problem(cvxpy.Minimize(cvxpy.norm1(A @ x - b)))
prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

                                     CVXPY                                     
                                    v1.1.12                                    
(CVXPY) May 19 12:47:20 AM: Your problem has 200 variables, 0 constraints, and 0 parameters.
(CVXPY) May 19 12:47:20 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 19 12:47:20 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 19 12:47:20 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 19 12:47:20 AM: Compiling problem (target solver=SCS).
(CVXPY) May 19 12:47:20 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing ->

726.7887441603591

In [11]:
# P-norm
np.random.seed(hash('p-norm') % 2 ** 31)

n = 20
m = int(n / 4)

G = np.random.randn(m, n)
f = np.random.randn(m)

power = np.pi
x = cvxpy.Variable(n)
constraints = [G @ x == f]
prob = cvxpy.Problem(cvxpy.Minimize(cvxpy.norm(x, power)), constraints)

prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

                                     CVXPY                                     
                                    v1.1.12                                    
(CVXPY) May 19 12:47:26 AM: Your problem has 20 variables, 1 constraints, and 0 parameters.
(CVXPY) May 19 12:47:26 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 19 12:47:26 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 19 12:47:26 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 19 12:47:26 AM: Compiling problem (target solver=SCS).
(CVXPY) May 19 12:47:26 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> 

0.27613646608591386

In [12]:
# L1-regularized Logistic regression
np.random.seed(hash('log-reg') % 2 ** 31)

p = 1000   # features
q = 10 * p  # total samples

w_true = np.random.randn(p, 1)
X_tmp = np.random.randn(p, q)

ips = -w_true.T.dot(X_tmp)
ps = (np.exp(ips) / (1 + np.exp(ips))).T
labels = 2 * (np.random.rand(q, 1) < ps) - 1

X_pos = X_tmp[:,np.where(labels==1)[0]]
X_neg = X_tmp[:,np.where(labels==-1)[0]]
X = np.hstack([X_pos, -X_neg]) # include labels with data

lam = 2
w = cvxpy.Variable(p)
obj = (cvxpy.sum(cvxpy.log_sum_exp(cvxpy.vstack([np.zeros((q,)), w.T @ X]), axis = 0))
       + lam * cvxpy.norm(w, 1))

prob = cvxpy.Problem(cvxpy.Minimize(obj))
prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

                                     CVXPY                                     
                                    v1.1.12                                    
(CVXPY) May 19 12:47:27 AM: Your problem has 1000 variables, 0 constraints, and 0 parameters.
(CVXPY) May 19 12:47:27 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 19 12:47:27 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 19 12:47:27 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 19 12:47:27 AM: Compiling problem (target solver=SCS).
(CVXPY) May 19 12:47:27 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -

1018.113179194932

In [13]:
# Matrix completion
np.random.seed(hash('matrix-completion') % 2 ** 31)

m = 100
n = 50

M = np.random.randn(m * n)
n_missing = int(0.8 * m * n)
missing_idx = np.random.permutation(m * n)[:n_missing]
valid_idx = np.setdiff1d(np.arange(m * n), missing_idx)

M[missing_idx] = 0.
X = cvxpy.Variable(m * n)

lam = 0.5
diff = cvxpy.reshape(X, (m, n)) - np.reshape(M, (m, n))
obj = cvxpy.norm(diff, "nuc") + lam * cvxpy.sum_squares(X)
constraints = [X[valid_idx] == M[valid_idx]]

prob = cvxpy.Problem(cvxpy.Minimize(obj), constraints)

prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

                                     CVXPY                                     
                                    v1.1.12                                    
(CVXPY) May 19 12:49:12 AM: Your problem has 5000 variables, 1 constraints, and 0 parameters.
(CVXPY) May 19 12:49:12 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 19 12:49:12 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 19 12:49:12 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 19 12:49:12 AM: Compiling problem (target solver=SCS).
(CVXPY) May 19 12:49:12 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -

784.0702371446

In [14]:
# Min-norm
np.random.seed(hash('min-norm') % 2 ** 31)

m = 500
n = int(m / 2)
A = np.random.randn(m, n)
b = 10 * np.random.randn(m)
G = 2 * np.random.randn(2 * n, n)

x = cvxpy.Variable(n)
obj = cvxpy.norm(A @ x - b)
constraints = [cvxpy.norm(G @ x) <= 1]

prob = cvxpy.Problem(cvxpy.Minimize(obj), constraints)
prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

                                     CVXPY                                     
                                    v1.1.12                                    
(CVXPY) May 19 12:49:13 AM: Your problem has 250 variables, 1 constraints, and 0 parameters.
(CVXPY) May 19 12:49:13 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 19 12:49:13 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 19 12:49:13 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 19 12:49:13 AM: Compiling problem (target solver=SCS).
(CVXPY) May 19 12:49:13 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing ->

225.15309253421773

In [15]:
# Lyapunov stability
np.random.seed(hash('lyapunov') % 2 ** 31)

n = 100
A = np.diag(-np.logspace(-0.5, 1, n))
U = scipy.linalg.orth(np.random.randn(n,n))
A = U.T.dot(A.dot(U))

P = cvxpy.Variable((n, n), symmetric=True)
obj = cvxpy.trace(P)
constraints = [A.T @ P + P @ A << -np.eye(n), P >> np.eye(n)]

prob = cvxpy.Problem(cvxpy.Minimize(obj), constraints)
# WARNING: this may run for a very long time (~10min)!
prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

                                     CVXPY                                     
                                    v1.1.12                                    
(CVXPY) May 19 12:49:13 AM: Your problem has 10000 variables, 2 constraints, and 0 parameters.
(CVXPY) May 19 12:49:13 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 19 12:49:13 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 19 12:49:13 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 19 12:49:13 AM: Compiling problem (target solver=SCS).
(CVXPY) May 19 12:49:13 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing 

103.81929726779265

In [16]:
# Portfolio
np.random.seed(hash('portfolio') % 2 ** 31)

m = 500
n = 5000
density = 0.1

mu = np.exp(0.01 * np.random.randn(n)) - 1.  # returns
D = np.random.rand(n) / 10.                  # idiosyncratic risk
F = scipy.sparse.rand(n, m, density) / 10.   # factor model

lambda_risk = 1
leverage = 1

x = cvxpy.Variable(n)
obj = mu.T @ x - lambda_risk * (cvxpy.sum_squares(F.T @ x) +
                                cvxpy.sum_squares(cvxpy.multiply(D, x)))

constraints = [cvxpy.sum(x) == leverage, x >= 0]

prob = cvxpy.Problem(cvxpy.Maximize(obj), constraints)
prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

                                     CVXPY                                     
                                    v1.1.12                                    
(CVXPY) May 19 12:57:26 AM: Your problem has 5000 variables, 2 constraints, and 0 parameters.
(CVXPY) May 19 12:57:26 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 19 12:57:26 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 19 12:57:26 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 19 12:57:26 AM: Compiling problem (target solver=SCS).
(CVXPY) May 19 12:57:26 AM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> Con

0.014456207545638183

In [17]:
# Sparse covariance estimation
np.random.seed(hash('cov-estim') % 2 ** 31)

num_samples = 10
n = 20
lam = 0.1

A = scipy.sparse.rand(n, n, 0.01)
A = A.T.dot(A).todense() + 0.1 * np.eye(n)
L = np.linalg.cholesky(np.linalg.inv(A))
X = L.dot(np.random.randn(n, num_samples))  # Draw m experiments according to the covariance matrix A^-1
S = X.dot(X.T) / num_samples                # Estimate of covariance matrix
mask = np.ones((n, n)) - np.eye(n)

theta = cvxpy.Variable((n, n))

obj = lam * cvxpy.norm1(cvxpy.multiply(mask, theta)) + cvxpy.trace(S @ theta) - cvxpy.log_det(theta)
prob = cvxpy.Problem(cvxpy.Minimize(obj))

prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

                                     CVXPY                                     
                                    v1.1.12                                    
(CVXPY) May 19 12:57:27 AM: Your problem has 400 variables, 0 constraints, and 0 parameters.
(CVXPY) May 19 12:57:27 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 19 12:57:27 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 19 12:57:27 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 19 12:57:27 AM: Compiling problem (target solver=SCS).
(CVXPY) May 19 12:57:27 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing ->

31.138074064273617

In [18]:
# Fused Lasso
np.random.seed(hash('fused-lasso') % 2 ** 31)

m = 1000
ni = 10
k = 1000
rho = 0.05
sigma = 0.05

A = np.random.randn(m, ni * k)
A /= np.sqrt(np.sum(A ** 2, 0))

x0 = np.zeros(ni * k)
for i in range(k):
    if np.random.rand() < rho:
        x0[i * ni:(i + 1) * ni] = np.random.rand()

b = A.dot(x0) + sigma * np.random.randn(m)
lam = 0.1 * sigma * np.sqrt(m * np.log(ni * k))

x = cvxpy.Variable(ni * k)
obj = cvxpy.sum_squares(A @ x - b) + lam * cvxpy.norm1(x) + lam * cvxpy.tv(x)
prob = cvxpy.Problem(cvxpy.Minimize(obj))

# WARNING: this may run for a long time (~2min)!
prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

                                     CVXPY                                     
                                    v1.1.12                                    
(CVXPY) May 19 12:57:30 AM: Your problem has 10000 variables, 0 constraints, and 0 parameters.
(CVXPY) May 19 12:57:30 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 19 12:57:30 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 19 12:57:30 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 19 12:57:30 AM: Compiling problem (target solver=SCS).
(CVXPY) May 19 12:57:30 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing 

108.82945575911212

In [19]:
# SVM
np.random.seed(hash('svm') % 2 ** 31)

m = 150
n = 500
A = np.random.randn(m, n)
x0 = np.random.rand(n)
y = np.sign(A.dot(x0) + 0.05 * np.random.randn(m))

lam = 1.0
x = cvxpy.Variable(n)
obj = (1./m) * cvxpy.sum(cvxpy.pos(1 - cvxpy.multiply(y, A @ x))) + lam * cvxpy.norm(x, 1)

prob = cvxpy.Problem(cvxpy.Minimize(obj))
prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

                                     CVXPY                                     
                                    v1.1.12                                    
(CVXPY) May 19 12:59:50 AM: Your problem has 500 variables, 0 constraints, and 0 parameters.
(CVXPY) May 19 12:59:50 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 19 12:59:50 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 19 12:59:50 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 19 12:59:50 AM: Compiling problem (target solver=SCS).
(CVXPY) May 19 12:59:50 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing ->

1.000921992633833

In [20]:
# Robust PCA
np.random.seed(hash('robust-pca') % 2 ** 31)

n = 100
r = 10 # Rank
density = 0.1

L0 = np.random.randn(n, r).dot(np.random.randn(r, n))  # Low rank matrix
S0 = scipy.sparse.rand(n, n, density)                  # Sparse matrix w/ Normally distributed entries.
S0.data = 10 * np.random.randn(len(S0.data))

M = L0 + S0

L = cvxpy.Variable((n, n))
S = cvxpy.Variable((n, n))

lam = 0.1
obj = cvxpy.norm(L, "nuc") + lam * cvxpy.norm1(S)
constraints = [L + S == M]

prob = cvxpy.Problem(cvxpy.Minimize(obj), constraints)
prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

                                     CVXPY                                     
                                    v1.1.12                                    
(CVXPY) May 19 12:59:51 AM: Your problem has 20000 variables, 1 constraints, and 0 parameters.
(CVXPY) May 19 12:59:51 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 19 12:59:51 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 19 12:59:51 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 19 12:59:51 AM: Compiling problem (target solver=SCS).
(CVXPY) May 19 12:59:51 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing 

1773.7217014659122

In [22]:
# Infinite push
np.random.seed(hash('infinite-push') % 2 ** 31)

m = 100
n = 200
d = 20
np.random.seed(0)

Xp = np.random.randn(m, d)
Xn = np.random.randn(n, d)

lam = 1
theta = cvxpy.Variable((d, 1))
Z = cvxpy.maximum(1 - (Xp @ theta @ np.ones((1, n)) - (Xn @ theta @ np.ones((1, m))).T), 0)
obj = cvxpy.max(cvxpy.sum(Z, axis=0)) + lam * cvxpy.sum_squares(theta)
prob = cvxpy.Problem(cvxpy.Minimize(obj))

prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

                                     CVXPY                                     
                                    v1.1.12                                    
(CVXPY) May 19 12:59:57 AM: Your problem has 20 variables, 0 constraints, and 0 parameters.
(CVXPY) May 19 12:59:57 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 19 12:59:57 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 19 12:59:57 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 19 12:59:57 AM: Compiling problem (target solver=SCS).
(CVXPY) May 19 12:59:57 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing -> 

100.0000424612099

In [23]:
# Quantile regression
np.random.seed(hash('quantile-regression') % 2 ** 31)

m = 100  # Number of data entries
n = 5    # Number of weights
k = 20   # Number of quantiles
p = 1
sigma = 0.1

x = np.random.rand(m) * 2 * np.pi * p
y = np.sin(x) + sigma * np.sin(x) * np.random.randn(m)
alphas = np.linspace(1. / (k + 1), 1 - 1. / (k + 1), k) # Do a bunch of quantiles at once

# RBF (Radial Basis Function) features
mu_rbf = np.array([np.linspace(-1, 2 * np.pi * p + 1, n)])
mu_sig = (2 * np.pi * p + 2)/n
X = np.exp(-(mu_rbf.T - x).T ** 2 / (2 * mu_sig ** 2)) # Gaussian

theta = cvxpy.Variable((n, k))

def quantile_loss(alphas, theta, X, y):
    m, n = X.shape
    k = len(alphas)
    Y = np.tile(y.flatten(), (k, 1)).T
    A = np.tile(alphas, (m, 1))
    Z = X @ theta - Y
    return cvxpy.sum(
        cvxpy.maximum(
            cvxpy.multiply(-A, Z),
            cvxpy.multiply(1 - A, Z)))

obj = quantile_loss(alphas, theta, X, y)
constraints = [X @ (theta[:, 1:] - theta[:, :-1]) >= 0]
prob = cvxpy.Problem(cvxpy.Minimize(obj), constraints)

prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

                                     CVXPY                                     
                                    v1.1.12                                    
(CVXPY) May 19 12:59:59 AM: Your problem has 100 variables, 1 constraints, and 0 parameters.
(CVXPY) May 19 12:59:59 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 19 12:59:59 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 19 12:59:59 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 19 12:59:59 AM: Compiling problem (target solver=SCS).
(CVXPY) May 19 12:59:59 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing ->

31.00550114596583

In [24]:
# Huber regression
np.random.seed(hash('huber-regression') % 2 ** 31)

m = 5000
n = 200

x0 = np.random.randn(n)
A = np.random.randn(m, n)
b = A.dot(x0) + 0.01 * np.random.randn(m)
# Add outlier noise.
k = int(0.02 * m)
idx = np.random.randint(m, size=k)
b[idx] += 10 * np.random.randn(k)

x = cvxpy.Variable(n)
prob = cvxpy.Problem(cvxpy.Minimize(cvxpy.sum(cvxpy.huber(A @ x - b))))

prob.solve(solver='SCS', verbose=True)
prob.solve(solver='SCS', verbose=True, acceleration_lookback=0)

                                     CVXPY                                     
                                    v1.1.12                                    
(CVXPY) May 19 01:00:04 AM: Your problem has 200 variables, 0 constraints, and 0 parameters.
(CVXPY) May 19 01:00:04 AM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 19 01:00:04 AM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 19 01:00:04 AM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 19 01:00:04 AM: Compiling problem (target solver=SCS).
(CVXPY) May 19 01:00:04 AM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffing ->

1390.1448274744562